In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys,os
from sklearn.preprocessing import normalize
from sklearn import datasets, linear_model
import warnings
warnings.filterwarnings('ignore')
sys.path.append('..')
sys.path.append('data')

%matplotlib inline
%load_ext autoreload
%autoreload 2
from IPython.display import clear_output,display,HTML
np.set_printoptions(suppress=True)
np.set_printoptions(precision=2)

In [2]:
data_dir = '../data'
data_fname = 'atoabsgovhack2017.xlsx'
data_fpath = os.path.join(data_dir,data_fname)
data = pd.read_excel(data_fpath,sheetname='Data')

In [3]:
df = data.set_index('Postcode')

measure, postcode, year, 
value, standard deviation, low, high
year from 2006, 2011, 2016, 2021, 2026, 2031
growth assumption for each population for each year

In [4]:
x_year = [
    'Income year',
]
# inflation rate = 0.025, interest rate = 0.015
growth_rates = np.array([0.025,0.015,0.015,0.015,0.025])
x_vars = [
    'Taxable income or loss1 ',
    #'Net tax',
    'Gross interest',
    'Net rent',
    'Net capital gain',
    'Total income or loss',
    #'Total deductions',
    #'Salary or wages',
    #'Medicare levy',
    #'Medicare levy surcharge',
    #'Total work related expenses',
    #'Gifts or donations',
    #'HELP assessment debt2'
]
y_vars = [
 '0-4 years',
 '5-9 years',
 '10-14 years',
 '15-19 years',
 '20-24 years',
 '25-29 years',
 '30-34 years',
 '35-39 years',
 '40-44 years',
 '45-49 years',
 '50-54 years',
 '55-59 years',
 '60-64 years',
 '65-69 years',
 '70-74 years',
 '75-79 years',
 '80-84 years',
 '85-89 years',
 '90-94 years',
 '95-99 years',
 '100 years and over',
 'Oceanian',
 'North-West European',
 'Southern and Eastern European',
 'North African and Middle Eastern',
 'South-East Asian',
 'North-East Asian',
 'Southern and Central Asian',
 'People of the Americas',
 'Sub-Saharan African',
 'Male',
 'Female',
 'under or 0',
 '$1 to 1000',
 '1000-1999',
 '2000+',
 'Married in a registered marriage',
 'Married in a de facto marriage',
 'Not married'
]
normalising_var = ['Individuals1']
new_years = [2020,2025,2030]

In [7]:
def ez_predict(df,postcode,y_var):
    """
    Predicts the measure values for 2021,2026,2031
    df : dataframe, indexed by postcode
    postcode : int
    y_var : string of name of measure you want to predict
    """
    x1 = np.divide(
        df.loc[postcode,x_vars],
        df.loc[postcode,normalising_var]
    ).values
    x2 = df.loc[postcode,x_year].values
    if x1.ndim == 1:
        X = np.concatenate((x1,x2),axis=0)
        final_x_vals = np.divide(
            df.loc[postcode,x_vars].values,
            df.loc[postcode,normalising_var].values
        )
    else:
        X = np.concatenate((x1,x2),axis=1)
        final_x_vals = np.divide(
            df.loc[postcode,x_vars].iloc[-1,:].values,
            df.loc[postcode,normalising_var].iloc[-1,:].values
        )
    Y = np.reshape(df.loc[postcode,y_var],(-1,1))
    regr = linear_model.LinearRegression()
    regr.fit(X, Y)

    x1_new = np.array(
        (
            final_x_vals * (growth_rates + 1)**5,
            final_x_vals * (growth_rates + 1)**10,
            final_x_vals * (growth_rates + 1)**15
        )
    )
    x2_new = np.reshape(np.array(new_years),(-1,1))
    x_new = np.concatenate((x1_new,x2_new),axis=1)
    pred = regr.predict(X=x_new)
    return pred

In [ ]:
pred = ez_predict(df,800,'Not married')

In [ ]:
pred_list = []
   
for postcode in df.index.unique():
    y_pred = []
    for y_var in y_vars:
        x1 = np.divide(
            df.loc[postcode,x_vars],
            df.loc[postcode,normalising_var]
        ).values
        x2 = df.loc[postcode,x_year].values
        if x1.ndim == 1:
            X = np.concatenate((x1,x2),axis=0)
            final_x_vals = np.divide(
                df.loc[postcode,x_vars].values,
                df.loc[postcode,normalising_var].values
            )
        else:
            X = np.concatenate((x1,x2),axis=1)
            final_x_vals = np.divide(
                df.loc[postcode,x_vars].iloc[-1,:].values,
                df.loc[postcode,normalising_var].iloc[-1,:].values
            )
        Y = np.reshape(df.loc[postcode,y_var],(-1,1))
        regr = linear_model.LinearRegression()
        regr.fit(X, Y)

        x1_new = np.array(
            (
                final_x_vals * (growth_rates + 1)**5,
                final_x_vals * (growth_rates + 1)**10,
                final_x_vals * (growth_rates + 1)**15
            )
        )
        x2_new = np.reshape(np.array(new_years),(-1,1))
        x_new = np.concatenate((x1_new,x2_new),axis=1)
        pred = regr.predict(X=x_new)
        y_pred.append(pred)
    pred_list.append(y_pred)

In [ ]:
measure_arr_list = []
for i,y_var in enumerate(y_vars):
    new_arr = []
    for j,postcode in enumerate(df.index.unique()):
        for year in new_years:
            pred = pred_list_final[i][j]
            new_rows = np.concatenate(
                (
                    np.reshape([postcode]*3,(-1,1)),
                    np.reshape(np.array(new_years),(-1,1)),
                    pred
                ),
                axis=1
            )
            new_arr.append(new_rows)
    measure_arr = pd.DataFrame(np.concatenate(new_arr,axis=0)).set_index([0,1])
    measure_arr_list.append(measure_arr)
out_df = pd.concat(measure_arr_list,axis=1).reset_index()
out_df.columns = ['postcode','year'] + y_vars

In [ ]:
out_df.to_csv('../data/predictions.csv')